<a href="https://colab.research.google.com/github/Bite-IT/agro-bursa-cli-parser/blob/main/PlayLy_%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D0%BC_%D1%81_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Повышаем качество распознавания OpenPose

In [ ]:
#@title ##**Устанавливаем необходимые библиотеки** { display-mode: "form" }
!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
!pip install torchvision==0.5
!pip install torch==1.4

from IPython.display import clear_output
from google.colab import files
import imageio
import youtube_dl
import cv2
import os
import torch
import fastai
import ffmpeg
import os.path as osp
import logging
import shutil
import re
import gc
from PIL import Image
from tqdm import tqdm
from os import path
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
import sys
import glob
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
%matplotlib inline

!rm -rf sample_data
clear_output()

In [ ]:
#@title ###*Скачиваем видео* { display-mode: "form" }
#@markdown *Вставьте ссылку на видео (YouTube or Twitter), или оставьте поле **source_url** пустым и загрузите файл с компьютера.*
source_url = 'https://www.youtube.com/watch?v=DeQwRF6FKQo' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *НЕ рекомендуется загружать длинные видео.*

#@markdown *Если будут ошибки - перезапустите клетку*

In [ ]:
#@title ###*Обрезаем видео* { display-mode: "form" }

#@markdown **Enter the time frame for croping the video**
target_start = '00:5:00' #@param {type:"string"}
target_end = '00:5:03' #@param {type:"string"}

#@markdown Enter a new resolution of the video file for rescaling (*for example, 640 and 480*), or leave it blank to work with the original resolution
width = '' #@param {type:"string"}
height = '' #@param {type:"string"}

rescale = ""
if width != '' and height != '':
  rescale = f"-s {width}x{height}"

!ffmpeg -i downloaded_video.mp4 $rescale -ss $target_start -to $target_end new_target.mp4
clear_output()
!rm video.mp4
!mv new_target.mp4 video.mp4
#@markdown *Display video:*
display_video = False #@param {type:"boolean"}
if display_video == True:
  display(mpy.ipython_display("video.mp4", height=400))

In [ ]:
#@title ##**Улучшаем качество видео с EDVR** { display-mode: "form" }
%cd /content
!pip install numpy opencv-python lmdb pyyaml
!pip install tb-nightly future


In [ ]:
#@title ###*Создаем папки и скачиваем модель* { display-mode: "form" }
!git clone -b old_version https://github.com/xinntao/EDVR
%cd /content/EDVR/
!mkdir  experiments
%cd /content/EDVR/experiments
!mkdir  pretrained_models
%cd /content/EDVR/codes/models/archs/dcn
!python setup.py develop
%cd /content/EDVR/codes
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1SGVehpZt4WL_X8Jh6blyqmHpc8DdImgv' -O /content/EDVR/experiments/pretrained_models/EDVR_REDS_deblurcomp_L.pth
clear_output()

In [ ]:
#@title ###*Инициализируем функции* { display-mode: "form" }
import utils.util as util
import data.util as data_util
import models.archs.EDVR_arch as EDVR_arch

workfolder = Path('./video')
source_folder = workfolder / "source"
inframes_root = workfolder / "inframes"
audio_root = workfolder / "audio"
outframes_root = workfolder / "outframes"
result_folder = workfolder / "result"
pretrained_models = Path('../experiments/pretrained_models')

def clean_mem():
    # torch.cuda.empty_cache()
    gc.collect()

def get_fps(source_path: Path) -> str:
    print(source_path)
    probe = ffmpeg.probe(str(source_path))
    stream_data = next(
        (stream for stream in probe['streams'] if stream['codec_type'] == 'video'),
        None,
    )
    return stream_data['avg_frame_rate']

def preProcess(imag_path_l, multiple):
  '''Need to resize images for blurred model (needs to be multiples of 16)'''
  for img_path in imag_path_l:
    im = Image.open(img_path)
    h, w = im.size
    # resize so they are multiples of 4 or 16 (for blurred)
    h = h - h % multiple
    w = w - w % multiple
    im = im.resize((h,w))
    im.save(img_path)

def purge_images(dir):
  for f in os.listdir(dir):
    if re.search('.*?\.jpg', f):
      os.remove(os.path.join(dir, f))

def extract_raw_frames(source_path: Path):
    inframes_folder = inframes_root / (source_path.stem)
    inframe_path_template = str(inframes_folder / '%5d.jpg')
    inframes_folder.mkdir(parents=True, exist_ok=True)
    purge_images(inframes_folder)
    ffmpeg.input(str(source_path)).output(
        str(inframe_path_template), format='image2', vcodec='mjpeg', qscale=0
    ).run(capture_stdout=True)

def make_subfolders(img_path_l, chunk_size):
  i = 0
  subFolderList = []
  source_img_path = Path('/content/EDVR/codes/video/inframes/video_subfolders')
  source_img_path.mkdir(parents=True, exist_ok=True)
  for img in img_path_l:
    if i % chunk_size == 0:
      img_path = source_img_path / str(i)
      img_path.mkdir(parents=True, exist_ok=True)
      subFolderList.append(str(img_path))
    i+=1
    img_name = osp.basename(img)
    img_path_name = img_path / img_name
    shutil.copyfile(img, img_path_name)

  return subFolderList

def remove_subfolders():
  shutil.rmtree('/content/EDVR/codes/video/inframes/video_subfolders', ignore_errors=True, onerror=None)

def edvrPredict(data_mode, chunk_size, stage):
  device = torch.device('cuda')
  os.environ['CUDA_VISIBLE_DEVICES'] = '0'
  data_mode = "blur_comp"
  stage = stage  # 1 or 2, use two stage strategy for REDS dataset.
  flip_test = False
  model_path = pretrained_models / 'EDVR_REDS_deblurcomp_L.pth'   

  print('Model Used: ', model_path)
  
  if data_mode == 'Vid4':
      N_in = 7  # use N_in images to restore one HR image
  else:
      N_in = 5

  predeblur, HR_in = False, False
  back_RBs = 40
  if data_mode == 'blur' or data_mode == 'blur_comp':
      predeblur, HR_in = True, True
  if stage == 2:
      HR_in = True
      back_RBs = 20
  if data_mode == 'TOF':
    model = TOF_arch.TOFlow(adapt_official=True)
  else:
    model = EDVR_arch.EDVR(128, N_in, 8, 5, back_RBs, predeblur=predeblur, HR_in=HR_in)

  #### dataset
  test_dataset_folder = '/content/EDVR/codes/video/inframes'

  #### evaluation
  crop_border = 0
  border_frame = N_in // 2  # border frames when evaluate
  # temporal padding mode
  if data_mode in ('Vid4','sharp_bicubic'):
      padding = 'new_info'
  else:
      padding = 'replicate'
  save_imgs = True

  save_folder = '/content/EDVR/codes/video/outframes'
  util.mkdirs(save_folder)

  #### set up the models
  model.load_state_dict(torch.load(model_path), strict=True)
  model.eval()
  model = model.to(device)

  avg_psnr_l, avg_psnr_center_l, avg_psnr_border_l = [], [], []
  subfolder_name_l = []
  # remove old video_subfolder if exists
  remove_subfolders()
  subfolder_l = sorted(glob.glob(osp.join(test_dataset_folder, '*')))

  # for each subfolder
  for subfolder in subfolder_l:
      subfolder_name = osp.basename(subfolder)
      subfolder_name_l.append(subfolder_name)
      save_subfolder = osp.join(save_folder, subfolder_name)

      img_path_l = sorted(glob.glob(osp.join(subfolder, '*')))
      if save_imgs:
          util.mkdirs(save_subfolder)
          purge_images(save_subfolder)

      # preprocess images (needed for blurred models)
      if predeblur:
        preProcess(img_path_l, 16)
      else:
        preProcess(img_path_l, 4)
      # make even more subfolders
      subFolderList = make_subfolders(img_path_l, chunk_size)

      #### read LQ and GT images in chunks of 1000
      for subSubFolder in subFolderList:
        clean_mem()
        imgs_LQ = data_util.read_img_seq(subSubFolder)
        subSubFolder_l = sorted(glob.glob(osp.join(subSubFolder, '*')))
        max_idx = len(subSubFolder_l)
        avg_psnr, avg_psnr_border, avg_psnr_center, N_border, N_center = 0, 0, 0, 0, 0

        # process each image
        for img_idx, img_path in tqdm(enumerate(subSubFolder_l)):
            img_name = osp.splitext(osp.basename(img_path))[0]
            select_idx = data_util.index_generation(img_idx, max_idx, N_in, padding=padding)
            imgs_in = imgs_LQ.index_select(0, torch.LongTensor(select_idx)).unsqueeze(0).to(device)

            if flip_test:
                output = util.flipx4_forward(model, imgs_in)
            else:
                output = util.single_forward(model, imgs_in)
            output = util.tensor2img(output.squeeze(0))

            if save_imgs:
                cv2.imwrite(osp.join(save_subfolder, '{}.jpg'.format(img_name)), output)
                # print('Saved Image:', str(osp.join(save_subfolder, '{}.jpg'.format(img_name))))

def moveProcessedFrames():
  shutil.rmtree('/content/EDVR/codes/video/inframes')
  os.rename('/content/EDVR/codes/video/outframes', '/content/EDVR/codes/video/inframes')

def build_video(source_path: Path) -> Path:
        out_path = result_folder / (
            source_path.name.replace('.mp4', '_no_audio.mp4')
        )
        outframes_folder = outframes_root / (source_path.stem)
        outframes_path_template = str(outframes_folder / '%5d.jpg')
        out_path.parent.mkdir(parents=True, exist_ok=True)
        if out_path.exists():
            out_path.unlink()
        fps = get_fps(source_path)
        print('Original FPS is: ', fps)

        ffmpeg.input(
            str(outframes_path_template),
            format='image2',
            vcodec='mjpeg',
            framerate=fps,
        ).output(str(out_path), crf=17, vcodec='libx264').run(capture_stdout=True)

        result_path = result_folder / source_path.name
        if result_path.exists():
            result_path.unlink()
        # making copy of non-audio version in case adding back audio doesn't apply or fails.
        shutil.copyfile(str(out_path), str(result_path))

        # adding back sound here
        audio_file = Path(str(source_path).replace('.mp4', '.aac'))
        if audio_file.exists():
            audio_file.unlink()

        os.system(
            'ffmpeg -y -i "'
            + str(source_path)
            + '" -vn -acodec copy "'
            + str(audio_file)
            + '"'
        )

        if audio_file.exists:
            os.system(
                'ffmpeg -y -i "'
                + str(out_path)
                + '" -i "'
                + str(audio_file)
                + '" -shortest -c:v copy -c:a aac -b:a 256k "'
                + str(result_path)
                + '"'
            )
        print('Video created here: ' + str(result_path))
        return result_path

def edvr_video(source_path: Path, chunk_size: int):
    # extract frames
    extract_raw_frames(source_path)

    # process frames
    edvrPredict("blur_comp", chunk_size, 1)

    # build back video
    build_video(source_path)
clear_output()

In [ ]:
#@title ###*Увеличиваем качество кадров* { display-mode: "form" }
%%time
edvr_video(Path('/content/video.mp4'), 100)
clear_output()

In [ ]:
#@title ###*Результат* { display-mode: "form" }
!rm -rf /content/video.mp4
!cp -r /content/EDVR/codes/video/result/video.mp4 /content/video.mp4
!cp -r /content/EDVR/codes/video/result/video.mp4 /content/enhanced_video.mp4
what_next = 'download' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/enhanced_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/enhanced_video.mp4')

In [ ]:
#@title ##**Определяем позы с OpenPose** { display-mode: "form" }
%cd /content
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


In [ ]:
#@title ###*Детектируем позы футболистов на улучшенном видео* { display-mode: "form" }
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video /content/enhanced_video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

In [ ]:
#@title ###*Детектируем позы футболистов на исходном  видео* { display-mode: "form" }
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video /content/video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output2.mp4

---
# <b>Увеличиваем разрешение видео

In [ ]:
#@title ##**Используем Video2X**{ display-mode: "form" }
!apt install ffmpeg
!apt install libmagic1 python3-yaml
!apt install libvulkan-dev
!pip install --user youtube-dl  

In [ ]:
#@title ###*Загружаем репозиторий Video2X c GitHub*{ display-mode: "form" }
!git clone https://github.com/k4yt3x/video2x.git

In [ ]:
#@title ###*Устанавливаем*{ display-mode: "form" }
import os
os.chdir('video2x/src')
!git checkout 4.7.0
!pip install -r requirements.txt
!rm -rf video2x.yaml
!wget -O video2x.yaml http://akas.io/v2xcolab
os.chdir('../..')

In [ ]:
#@title ###*Скачиваем модели*{ display-mode: "form" }
!wget https://github.com/nihui/realsr-ncnn-vulkan/releases/download/20200818/realsr-ncnn-vulkan-20200818-linux.zip
!7z x realsr-ncnn-vulkan-20200818-linux.zip
!wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20200818/waifu2x-ncnn-vulkan-20200818-linux.zip
!7z x waifu2x-ncnn-vulkan-20200818-linux.zip
!wget https://github.com/nihui/srmd-ncnn-vulkan/releases/download/20200818/srmd-ncnn-vulkan-20200818-linux.zip
!7z x srmd-ncnn-vulkan-20200818-linux.zip

In [ ]:
#@title ###*Запускаем обработку для video.mp4  (может занять > 10 часов)*{ display-mode: "form" }
!pip install -U PyYAML
!python video2x/src/video2x.py -i /content/video.mp4 -o video_4x.mp4 -d waifu2x_ncnn_vulkan -r 4.0

In [ ]:
#@title ###*Скачиваем результат на диск*{ display-mode: "form" }
!cp video_4x.mp4 "/content/drive/MyDrive/video_4x_2.mp4"

In [ ]:
#@title ##**Используем ESRGAN**{ display-mode: "form" }
%cd /content
!git clone https://github.com/xinntao/ESRGAN


In [ ]:
#@title ###*Загружаем и устанавливаем репозиторий*{ display-mode: "form" }

!cp -r /content/video.mp4 /content/ESRGAN/
%cd /content/ESRGAN
!git checkout tags/old-arch
model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models
import architecture as arch
import os.path
!mkdir frames
!rm -rf results/baboon_ESRGAN.png


In [ ]:
#@title ###*Делим видео на кадры* { display-mode: "form" }
frames_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [ ]:
#@title ###*Проверяем состояние GPU*{ display-mode: "form" }
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
#@title ###*Устанавливаем утилиты для работы с GPU*{ display-mode: "form" }
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [ ]:
#@title ###*Повышаем разрешение каждого кадра* { display-mode: "form" }
import architecture as arch
import os.path
upscale = 4 
%env CUDA_VISIBLE_DEVICES=0
device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

count_frames = 0

for path in glob.glob('frames/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    count_frames += 1
    
    print("Processed: {} из {}".format(str(count_frames), str(frames_of_video)))


In [ ]:
#@title ###*Собираем видео из кадров* { display-mode: "form" }
frames = []
img = os.listdir("results/")
img.sort()
for i in img:
  frames.append(imageio.imread("results/"+i))
frames = np.array(frames)
imageio.mimsave("upscaled_video.mp4", frames, fps=fps_of_video)

print('Сборка завершена')
!cp -r upscaled_video.mp4 /content/upscaled_video.mp4
!cp -r upscaled_video.mp4 /content/video.mp4
clear_output()

In [ ]:
#@title ###*Получаем результат* { display-mode: "form" }
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/upscaled_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("/content/upscaled_video.mp4")

In [ ]:
#@title ###*Детектируем позы футболистов на видео с увеличенным разрешением Video2X* { display-mode: "form" }
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video /content/drive/MyDrive/video_4x_2.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

In [ ]:
#@title ###*Детектируем позы футболистов на видео с увеличенным разрешением ESRGAN* { display-mode: "form" }
!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video /content/upscaled_video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output2.mp4